In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()  # this loads the .env file automatically

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_BASE"]  = os.getenv("OPENAI_API_BASE")
os.environ["OPENAI_MODEL_NAME"] = os.getenv("OPENAI_MODEL_NAME")
os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY")

In [3]:
from crewai import Agent, Task, Crew

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

In [5]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [6]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [7]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [8]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [10]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [11]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [12]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [13]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

In [14]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
manager_llm = ChatOpenAI(
    model="mistral-large-latest",  # or mistral-small, mistral-medium
    temperature=0.7,
    openai_api_base="https://api.mistral.ai/v1",
    openai_api_key=os.environ["OPENAI_API_KEY"]
),
    process=Process.hierarchical,
    verbose=True
)

In [15]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

In [16]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
Thought: I need to delegate the task of continuously monitoring and analyzing market data for AAPL to the Data Analyst. I should also ask the Trading Strategy Developer to help identify trends and predict market movements using statistical modeling and machine learning.

Action: Delegate work to co-worker
Action Input: {"coworker": "Data Analyst", "task": "Continuously monitor and analyze market data for AAPL. Use statistical modeling and machine learning to identify trends and predict market movements.", "context": "The task requires constant attention to market data for AAPL. You will need to use statistical modeling and machine learning to identify trends and predict market movements. Please provide insights an

In [17]:
from IPython.display import Markdown
Markdown(result)

---

**Risk Analysis Report: AAPL Trading using Moving Average Crossover Strategy**

**1. Executive Summary**

This report provides a comprehensive analysis of the potential risks associated with the Moving Average Crossover (MAC) strategy for trading Apple Inc. (AAPL) and suggests mitigation strategies to manage these risks effectively. By understanding and addressing these risks, traders can enhance the strategy's performance and align it with the firm's risk tolerance.

**2. Strategy Overview**

The Moving Average Crossover strategy involves using moving averages and the Relative Strength Index (RSI) to generate entry and exit signals for trading AAPL. The strategy employs a short-term moving average (SMA) and a long-term moving average (LMA) to identify potential uptrends (bullish crossover) and downtrends (bearish crossover).

**3. Potential Risks and Mitigation Strategies**

**3.1 False Signals in Choppy Markets**

*Risk*: Moving averages can generate false signals during periods of market volatility or consolidation, leading to poor trading decisions.

*Mitigation*:

- Use RSI to confirm signals. An RSI above 70 indicates overbought conditions, while an RSI below 30 indicates oversold conditions. This can help filter out false signals.
- Combine the MAC strategy with other technical indicators, such as the On-Balance Volume (OBV) or the Moving Average Convergence Divergence (MACD), to get a more comprehensive view of the market.

**3.2 Lagging Indicators**

*Risk*: Moving averages are lagging indicators, meaning they provide signals after the trend has already begun, potentially missing optimal entry and exit points.

*Mitigation*:

- Combine moving averages with leading indicators to get a more timely indication of trend changes.
- Consider using a shorter-term moving average, such as a 10-day or 20-day SMA, to reduce lag time.

**3.3 Whipsaw Markets**

*Risk*: In markets with no clear trend, moving averages can generate a large number of false signals, leading to multiple small losses.

*Mitigation*:

- Implement a filtering mechanism, such as only taking trades when the moving averages are trending in a clear direction over a longer timeframe.
- Use a wider stop-loss to account for market noise and reduce the impact of whipsaw markets.

**3.4 Over-Reliance on a Single Strategy**

*Risk*: Relying solely on the MAC strategy can lead to missed opportunities or increased risk if market conditions change.

*Mitigation*:

- Diversify trading strategies by incorporating fundamental analysis and other technical analysis tools to make more informed decisions.
- Regularly review and update the strategy to adapt to changing market conditions.

**3.5 Market Noise**

*Risk*: Short-term price fluctuations can cause moving averages to cross frequently, leading to numerous false signals.

*Mitigation*:

- Use longer-term moving averages to smooth out short-term noise. For example, consider using a 50-day and 200-day moving average for a clearer trend indication.
- Use a wider moving average window to reduce the impact of market noise.

**4. Risk Management**

Implementing strict risk management practices is crucial for managing the risks associated with the MAC strategy. This includes:

- Setting stop-loss orders to limit potential losses.
- Using a risk-to-reward ratio to ensure that the potential reward outweighs the risk.
- Monitoring market conditions and adjusting stop-loss levels as needed.

**5. Backtesting and Adaptive Strategy**

*Risk*: The strategy's performance may vary depending on market conditions.

*Mitigation*:

- Backtest the strategy using historical data to evaluate its performance and fine-tune the parameters.
- Be adaptive and ready to adjust the strategy based on changing market conditions. For example, during high volatility, use wider stop-losses or rely more on fundamental analysis.

**6. Conclusion**

By understanding and addressing the potential risks associated with the Moving Average Crossover strategy for AAPL trading, traders can enhance the strategy's performance and align it with the firm's risk tolerance. Implementing the suggested mitigation strategies, combining the strategy with other technical indicators, and maintaining a strong risk management framework are essential for effectively managing the risks associated with this popular trading approach.

---